<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/Streamlit_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q streamlit
!pip install -q pyngrok

In [3]:
%%writefile app.py

import types
import random
import dataclasses
import streamlit as st
from typing import TypeVar
from typing import List, Tuple
from string import ascii_lowercase

StateT = TypeVar('StateT')

def persistent_game_state(initial_state: StateT) -> StateT:
    session_id = st.report_thread.get_report_ctx().session_id
    session = st.server.server.Server.get_current()._get_session_info(session_id).session
    if not hasattr(session, '_gamestate'):
        setattr(session, '_gamestate', initial_state)
    return session._gamestate

PROTOTYPE = """
 ┏━━┑
 ┃  O>
 ┃>╦╧╦<
 ┃ ╠═╣
 ┃ ╨ ╨
 ┻━━━━
"""


STEPS = [
"""
 ┏━━┑
 ┃
 ┃
 ┃
 ┃
 ┻━━━━
""",
"""
 ┏━━┑
 ┃  O
 ┃
 ┃
 ┃
 ┻━━━━
""",
"""
 ┏━━┑
 ┃  O>
 ┃
 ┃
 ┃
 ┻━━━━
""",
"""
 ┏━━┑
 ┃  O>
 ┃ ╔╧╗
 ┃ ╚═╝
 ┃
 ┻━━━━
""",
"""
 ┏━━┑
 ┃  O>
 ┃>╦╧╗
 ┃ ╚═╝
 ┃
 ┻━━━━
""",
"""
 ┏━━┑
 ┃  O>
 ┃>╦╧╦<
 ┃ ╚═╝
 ┃
 ┻━━━━
""",
"""
 ┏━━┑
 ┃  O>
 ┃>╦╧╦<
 ┃ ╠═╝
 ┃ ╨
 ┻━━━━
""",
"""
 ┏━━┑
 ┃  O>
 ┃>╦╧╦<
 ┃ ╠═╣
 ┃ ╨ ╨
 ┻━━━━
"""
]

MIN_LENGTH = 2
MAX_LENGTH = 8

@st.cache
def get_words() -> List[str]:
    vocab = ['die', 'die', 'die', 'die']
    words = [w for w in vocab if MIN_LENGTH <= len(w) <= MAX_LENGTH]
    words = [w for w in words if all('a' <= c <= 'z' for c in w)]
    return words

@dataclasses.dataclass
class GameState:
    game_number: int
    word: str
    guessed: Tuple[str, ...] = ()
    step: int = 0
    game_over: bool = False

state = persistent_game_state(initial_state=GameState(0, random.choice(get_words()))) 

if st.button("new game"):
    state.guessed = ()
    state.step = 0
    state.game_number += 1
    state.word = random.choice(get_words())
    state.game_over = False

if not state.game_over:
    guess = st.text_input("Escolha uma letra", max_chars=1, key=state.game_number)

    if not guess:
        st.write("por favor, escolha")
    elif guess < 'a' or guess > 'z':
        st.write("por favor, escolha uma letra minúscula!")
    elif guess in state.guessed:
        st.write(f"você escolheu **{guess}**")
    elif guess not in state.word:
        st.write(f"a palavra não possui **{guess}**")
        state.step += 1
        state.guessed += (guess,)
    else: 
        st.write("boa escolha")
        state.guessed += (guess,)

if state.step == len(STEPS) - 1:
    st.markdown(f"perdeu, a palavra era  **{state.word}**")
    state.game_over = True
elif all(c in state.guessed for c in state.word):
    st.markdown(f"**Aeeeeeeeee! WinWin!**")
    state.game_over = True

# Show the chicken
st.text(STEPS[state.step])

# Show the word
chars = [c if c in state.guessed else "_" for c in state.word]
st.text(" ".join(chars))

# Show the guessed letters
st.text(f'escolhidas: {" ".join(state.guessed)}')

Writing app.py


In [4]:
!ls

app.py	sample_data


In [5]:
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [6]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

'http://211512ded51a.ngrok.io'

# Referências
> https://medium.com/@jcharistech/how-to-run-streamlit-apps-from-colab-29b969a1bdfc

> https://github.com/joelgrus/streamlit-games/blob/master/hangchicken.py